## 3.4.4
## Kevin Hahn

For this challenge I was asked to change the Support Vector Machine Regressor model to a Classifier model with 30 features. I used Random Forest to determine importance weights and re-ran the model as a classifier with those 30 features. 

The mean accuracy of the revised model was 0.9564, and a KFolds method with 5 folds had mean accuracies of 
[0.8267, 0.8241, 0.8226, 0.8276, 0.8269]. The revised feature list to classify recipes as "good" (i.e., mean rating of 3.5 or higher on a 0-5 scale) follows:  

['calories', 'sodium', 'fat', 'protein', 'gourmet', 'bon appétit', 'drink', 'quick & easy', 'summer', 'house & garden', 'vegetarian', 'bake', 'kid-friendly', 'winter', 'fall', 'onion', 'tree nut free', 'herb', 'wheat/gluten-free', 'gin', 'vegetable', 'dairy', 'fruit', 'spring', 'sauce', 'side', 'milk/cream', 'dinner', 'egg', 'peanut free']

In [13]:
import math
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
%matplotlib inline
import scipy
import sklearn
import seaborn as sns
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
%matplotlib inline
sns.set_style('white')

In [14]:
dataframe = pd.read_csv("~/Downloads/recipes/epi_r.csv")

names = dataframe.columns

In [15]:
# dropping calroies, protein, fat, sodium columns because a lot of NA
# dataframe = dataframe.drop(['calories','protein','fat','sodium'], 1)

In [ ]:
# good column is 1 if rating is 3.5 or higher average rating, and 0 if < 3.5 average rating
dataframe['good'] = dataframe['rating'].apply(lambda x: 1 if x > 3.499 else 0)

In [ ]:
## dropping calories, protein, fat, sodium columns because a lot of NA
# dataframe = dataframe.drop(['calories','protein','fat','sodium'], 1)

In [26]:
wordfrequency = dataframe.sum(axis=0, numeric_only=True)

In [36]:
wordfrequency.sort_values(ascending=False)
dataframe.groupby(["good"]).sum()

,rating,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,30 days of groceries,advance prep required,alabama,alaska,alcoholic,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
good,,,,,,,,,,,,,,,,,,,,,
0,6420.625,1.0,0.0,1.0,17.0,1.0,37.0,2.0,0.0,455.0,...,7.0,156.0,0.0,0.0,60.0,1.0,7.0,14.0,6.0,73.0
1,68061.875,5.0,1.0,16.0,10.0,6.0,72.0,1.0,2.0,380.0,...,18.0,372.0,1.0,6.0,238.0,2.0,0.0,14.0,13.0,383.0


In [48]:
# groupedgood.sort_values(ascending=False)
transposed = groupedgood.T

In [54]:
transposed.sort_values(by=0, ascending=False)

# make a ratio of bad to good features/words based on current features

good,0,1
rating,6420.625,68061.875
peanut free,1470.000,6920.000
bon appétit,1448.000,7907.000
soy free,1420.000,6668.000
vegetarian,1403.000,5443.000
gourmet,1255.000,5393.000
tree nut free,1234.000,5810.000
kosher,1152.000,5023.000
pescatarian,1145.000,4897.000
wheat/gluten-free,963.000,3943.000


In [56]:
transposed.sort_values(by=1, ascending=True)

good,0,1
nancy silverton,1.000,0.000
dorie greenspan,1.000,0.000
crêpe,1.000,0.000
frittata,2.000,0.000
fritter,1.000,0.000
germany,1.000,0.000
mississippi,1.000,0.000
pancake,5.000,0.000
waffle,1.000,0.000
meatloaf,2.000,0.000


In [17]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

dataframe = dataframe.dropna().copy()

# dropping calroies, protein, fat, sodium columns because a lot of NA

rfc = ensemble.RandomForestClassifier()
X = dataframe.drop(['title', 'rating', 'good'], 1)
Y = dataframe['good']
X = pd.get_dummies(X)

print(cross_val_score(rfc, X, Y, cv=10))

df = pd.DataFrame()
df['imp']= rfc.fit(X, Y).feature_importances_
df['features'] = X.columns

featuresDF = df.sort_values(by='imp', ascending=False)[0:30] #[0:30]

[ 0.83679899  0.83238815  0.83553875  0.83742911  0.83480454  0.82156368
  0.83732661  0.82912989  0.84110971  0.82912989]


In [32]:
newlist = []
for feature in featuresDF['features']:
    newlist.append(feature)
print(newlist)

['calories', 'sodium', 'fat', 'protein', 'gourmet', 'bon appétit', 'drink', 'quick & easy', 'summer', 'house & garden', 'vegetarian', 'bake', 'kid-friendly', 'winter', 'fall', 'onion', 'tree nut free', 'herb', 'wheat/gluten-free', 'gin', 'vegetable', 'dairy', 'fruit', 'spring', 'sauce', 'side', 'milk/cream', 'dinner', 'egg', 'peanut free']


In [36]:
testDF_X = dataframe[['calories',
 'sodium',
 'fat',
 'protein',
 'gourmet',
 'bon appétit',
 'drink',
 'quick & easy',
 'summer',
 'house & garden',
 'vegetarian',
 'bake',
 'kid-friendly',
 'winter',
 'fall',
 'onion',
 'tree nut free',
 'herb',
 'wheat/gluten-free',
 'gin',
 'vegetable',
 'dairy',
 'fruit',
 'spring',
 'sauce',
 'side',
 'milk/cream',
 'dinner',
 'egg',
 'peanut free']]
testDF_y = dataframe.good

In [42]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svc = SVC()
X = testDF_X
Y = testDF_y
svc.fit(X,Y)
print(svc.score(X, Y))
print(cross_val_score(svc, X, Y, cv=5))

0.956442259203
[ 0.82666246  0.82414119  0.8225654   0.82760794  0.82692308]
